In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import plotly.express as px 
%matplotlib inline

In [3]:
# Importing the House price dataset
df = pd.read_csv('../input/house-price-data/Bengaluru_House_Data.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [4]:
print('the shape of dataset is :',df.shape)

the shape of dataset is : (13320, 9)


## Data Cleaning

In [5]:
#Checking the null values
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [6]:
# Checking the dataset info
df.describe()

,bath,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [7]:
# Observing the Bath and Balcony column
y0 = df.bath
y1 = df.balcony
fig = go.Figure()
fig.add_trace(go.Box(y=y0,name='Bath',quartilemethod="linear"))
fig.add_trace(go.Box(y=y1,name='Balcony',quartilemethod="linear"))
fig.update_layout(title='Bath & Balcony', yaxis_title='Count',height=420,width=1080)
fig.show()

In [8]:
# As we can not fill the null data in Society by any means so saying it: No data (for nan value)
# As we say that maximum count of Bath and Balcony lies between its LQ - UQ: We can fill the bath,balcony by its mean
# For size we can not
df.bath = df.bath.fillna(int(df.bath.mean()))
df.balcony = df.balcony.fillna(int(df.balcony.mean()))
df.society = df.society.fillna("No data")
df = df[df['location'].notna()]
df = df[df['size'].notna()]
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,No data,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,No data,1200,2.0,1.0,51.00


In [9]:
# Again checking the null values
df.isnull().sum()

area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

In [10]:
# Removing un-necessary columns
df.drop(columns=['area_type','availability','society'],inplace=True)


### Cleaning columns : 'bath' and 'balcony'

In [11]:
# Size column having mixed value like '2 bhk or 2 bedroom' so i am generalising it
# Renaming column size to BHK
# Converting price into lakhs
df['size'] = df['size'].apply(lambda x: int(x.split(' ')[0]))
df.rename(columns={'size':'bhk'},inplace=True)
df['price'] = df['price']*100000
df.head()


,location,bhk,total_sqft,bath,balcony,price
0,Electronic City Phase II,2,1056,2.0,1.0,3907000.0
1,Chikka Tirupathi,4,2600,5.0,3.0,12000000.0
2,Uttarahalli,3,1440,2.0,3.0,6200000.0
3,Lingadheeranahalli,3,1521,3.0,1.0,9500000.0
4,Kothanur,2,1200,2.0,1.0,5100000.0


### Cleaning Column: total_sqft

In [12]:
#Checking total_sqft column entires
df.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

We can see that total_sqft column have some entires as (1113 - 1505) so now we have to check more carefully

In [13]:
# Making a function to have all different types of entires
def check_float(x):
    try:
        float(x)
    except:
        return False
    return True
df[~df['total_sqft'].apply(check_float)].head(10)

,location,bhk,total_sqft,bath,balcony,price
30,Yelahanka,4,2100 - 2850,4.0,0.0,18600000.0
56,Devanahalli,4,3010 - 3410,2.0,1.0,19200000.0
81,Hennur Road,4,2957 - 3450,2.0,1.0,22450000.0
122,Hebbal,4,3067 - 8156,4.0,0.0,47700000.0
137,8th Phase JP Nagar,2,1042 - 1105,2.0,0.0,5400500.0
165,Sarjapur,2,1145 - 1340,2.0,0.0,4349000.0
188,KR Puram,2,1015 - 1540,2.0,0.0,5680000.0
224,Devanahalli,3,1520 - 1740,2.0,1.0,7482000.0
410,Kengeri,1,34.46Sq. Meter,1.0,0.0,1850000.0
549,Hennur Road,2,1195 - 1440,2.0,0.0,6377000.0


In [14]:
# Now converting this type of entries ('1125 - 4112') into INT and storing their average
def dash_to_int(x):
    dash = x.split('-')
    if len(dash)==2:
        return (float(dash[0]) + float(dash[1]))/2
    try:
        return float(x)
    except:
        return None
df['total_sqft'] = df['total_sqft'].apply(dash_to_int)
df.rename(columns={'total_sqft':'total_area'},inplace=True)
df.head()

,location,bhk,total_area,bath,balcony,price
0,Electronic City Phase II,2,1056.0,2.0,1.0,3907000.0
1,Chikka Tirupathi,4,2600.0,5.0,3.0,12000000.0
2,Uttarahalli,3,1440.0,2.0,3.0,6200000.0
3,Lingadheeranahalli,3,1521.0,3.0,1.0,9500000.0
4,Kothanur,2,1200.0,2.0,1.0,5100000.0


In [15]:
# Cross checking total_area entries types
df[df['total_area'].apply(check_float)]

,location,bhk,total_area,bath,balcony,price
0,Electronic City Phase II,2,1056.0,2.0,1.0,3907000.0
1,Chikka Tirupathi,4,2600.0,5.0,3.0,12000000.0
2,Uttarahalli,3,1440.0,2.0,3.0,6200000.0
3,Lingadheeranahalli,3,1521.0,3.0,1.0,9500000.0
4,Kothanur,2,1200.0,2.0,1.0,5100000.0
...,...,...,...,...,...,...
13315,Whitefield,5,3453.0,4.0,0.0,23100000.0
13316,Richards Town,4,3600.0,5.0,1.0,40000000.0
13317,Raja Rajeshwari Nagar,2,1141.0,2.0,1.0,6000000.0
13318,Padmanabhanagar,4,4689.0,4.0,1.0,48800000.0


### Cleaning column: 'location'

In [16]:
df.head()

,location,bhk,total_area,bath,balcony,price
0,Electronic City Phase II,2,1056.0,2.0,1.0,3907000.0
1,Chikka Tirupathi,4,2600.0,5.0,3.0,12000000.0
2,Uttarahalli,3,1440.0,2.0,3.0,6200000.0
3,Lingadheeranahalli,3,1521.0,3.0,1.0,9500000.0
4,Kothanur,2,1200.0,2.0,1.0,5100000.0


In [17]:
# Applying strip function to stripping the location names
df['location'] = df['location'].apply(lambda x: x.strip())

In [18]:
# As we have a lot of locations name so i am reducing the name types by assigning: location occuring for less than 10times as OTHER
loc = df.location.value_counts()
loc[loc<=10].count()
location_lessthan_10 = loc[loc<=10]
df.location = df.location.apply(lambda x: 'other' if x in location_lessthan_10 else x)
df.head()



,location,bhk,total_area,bath,balcony,price
0,Electronic City Phase II,2,1056.0,2.0,1.0,3907000.0
1,Chikka Tirupathi,4,2600.0,5.0,3.0,12000000.0
2,Uttarahalli,3,1440.0,2.0,3.0,6200000.0
3,Lingadheeranahalli,3,1521.0,3.0,1.0,9500000.0
4,Kothanur,2,1200.0,2.0,1.0,5100000.0


In [19]:
# cross checking
df.location.value_counts()

other                        2882
Whitefield                    540
Sarjapur  Road                397
Electronic City               304
Kanakpura Road                273
                             ... 
2nd Phase Judicial Layout      11
Thyagaraja Nagar               11
Tindlu                         11
Banjara Layout                 11
HAL 2nd Stage                  11
Name: location, Length: 242, dtype: int64

#### ADDING a column named 'price_per_sqft' for understanding the data more clearly

In [20]:
# Making column 'price_per_sqft'
df['price_per_sqft'] = df['price']/df['total_area']

In [21]:
df.price_per_sqft.describe()

count    1.325700e+04
mean     7.912825e+03
std      1.064976e+05
min      2.678298e+02
25%      4.271186e+03
50%      5.438596e+03
75%      7.313318e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [22]:
df.shape

(13303, 7)

## Outliers removal by the knowledge of bussiness domain

#### Dataset must be Distributed uniformally
1. 1bhk house must be more than 300 sqft
2. Mostly price of 3 or 4 bhk house > price of 1bhk house
3. Number of bathrooms is less than count of BHK or (bhk+2) of house
 

In [23]:
df.describe()

,bhk,total_area,bath,balcony,price,price_per_sqft
count,13303.000000,13257.000000,13303.000000,13303.000000,1.330300e+04,1.325700e+04
mean,2.803728,1558.809175,2.689619,1.558295,1.125840e+07,7.912825e+03
std,1.295022,1238.478935,1.339393,0.807914,1.489938e+07,1.064976e+05
min,1.000000,1.000000,1.000000,0.000000,8.000000e+05,2.678298e+02
25%,2.000000,1100.000000,2.000000,1.000000,5.000000e+06,4.271186e+03
50%,3.000000,1275.000000,2.000000,2.000000,7.200000e+06,5.438596e+03
75%,3.000000,1680.000000,3.000000,2.000000,1.200000e+07,7.313318e+03
max,43.000000,52272.000000,40.000000,3.000000,3.600000e+08,1.200000e+07


### Normalising the data set by taking 'price_per_sqft' as a feature

In [25]:
## Removing general outliers and making dataset distributed normally
def normally_dist(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        red_df = subdf[(subdf.price_per_sqft>(m-st)) &(subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,red_df],ignore_index = True)
    return df_out
df = normally_dist(df)
df.shape


(8369, 7)

In [26]:
df.describe()

,bhk,total_area,bath,balcony,price,price_per_sqft
count,8369.000000,8369.000000,8369.000000,8369.000000,8.369000e+03,8369.000000
mean,2.638667,1489.853828,2.520134,1.571275,9.023334e+06,5751.929985
std,1.047813,902.712226,1.101280,0.794019,8.086565e+06,2307.517130
min,1.000000,300.000000,1.000000,0.000000,1.000000e+06,2439.024390
25%,2.000000,1100.000000,2.000000,1.000000,5.000000e+06,4316.455696
50%,2.000000,1270.000000,2.000000,2.000000,6.800000e+06,5205.431755
75%,3.000000,1626.000000,3.000000,2.000000,9.926000e+06,6422.764228
max,27.000000,30400.000000,27.000000,3.000000,2.100000e+08,26000.000000


In [27]:
# Checking total_area data points
fig = px.box(df,y='total_area',
            title='Total Area')
fig.update_layout(yaxis_title ='Area')
fig.show()

In [28]:
# 1bhk house must be more than 300 sqft
df = df[~(df.total_area/df.bhk < 300)]
df.shape

(8142, 7)

In [29]:
## Viz where it is happening that 2bhk price > 3bhk

def plot_scatter(df,location):
    bhk2 = df[(df.location == location) &(df.bhk ==2)]
    bhk3 = df[(df.location == location) &(df.bhk ==3)]
    fig = go.Figure() 
    fig.add_trace(go.Scatter(x=bhk2.total_area,y=bhk2.price,
                    mode='markers',
                    name='2 BHK'))
    fig.add_trace(go.Scatter(x=bhk3.total_area,y=bhk3.price,
                    mode='markers',
                    name='3 BHK'))
    fig.update_layout(title='2BHK vs 3BHK Price', xaxis_title='Area in Sqft', yaxis_title='Price',height=620,width=680)
    fig.show()

plot_scatter(df,'Rajaji Nagar')



In [30]:
# new feature inspection = Bathrooms (i went to my bussines manager)
df.bath.unique()

array([ 3.,  2.,  4.,  5.,  8.,  1.,  6.,  7.,  9., 12., 16., 13.])

In [31]:
y0 = df.bath
y1 = df.bhk
fig = go.Figure()
fig.add_trace(go.Box(y=y0,name='Bath',quartilemethod="linear"))
fig.add_trace(go.Box(y=y1,name='BHK',quartilemethod="linear"))
fig.update_layout(title='Bath & BHK', yaxis_title='Count',height=420,width=1080)
fig.show()

In [32]:
## As per BM i know that their can not be bathrooms more than Count of BHK +2 (may be outlier)

In [33]:
df[df['bath']> df['bhk']+2]

,location,bhk,total_area,bath,balcony,price,price_per_sqft
1231,Chikkabanavar,4,2460.0,7.0,2.0,8000000.0,3252.032520
3760,Nagasandra,4,7000.0,8.0,1.0,45000000.0,6428.571429
4905,Thanisandra,3,1806.0,6.0,2.0,11600000.0,6423.034330
6423,other,6,11338.0,9.0,1.0,100000000.0,8819.897689


In [34]:
# Removing bath outliers
df.drop(df[df['bath']> df['bhk']+2].index, inplace = True)

In [35]:
y0 = df.total_area
y1 = df.bhk
fig = go.Figure()
fig.add_trace(go.Box(y=y0,name='total_area',quartilemethod="linear"))
fig.add_trace(go.Box(y=y1,name='BHK',quartilemethod="linear"))
fig.update_layout(title='Bath & BHK', yaxis_title='Count',height=420,width=1080)
fig.show()

In [36]:
df.to_csv('Final_cleaned_data_4.csv')

## AS WE CLEANED OUR DATA NOW I AM MOVING TOWARDS MODEL TRAINING

In [37]:
dff = pd.read_csv('../input/cleaned-bangalore-house-data-set/Final_cleaned_data_4.csv',index_col=0)

In [38]:
dff.shape

(8138, 7)

In [39]:
# one hot enchoding
dummies_dff =  pd.get_dummies(dff.location)
dff1 = pd.concat([dff,dummies_dff],axis='columns')

In [40]:
#dummmy variable trap avoided and string categorical data column removed

dff = dff1.drop(columns=['Vittasandra','location'])

In [41]:
dff.head(3)

,bhk,total_area,bath,balcony,price,price_per_sqft,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,3,1630.0,3.0,2.0,19400000.0,11901.840491,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1875.0,2.0,3.0,23500000.0,12533.333333,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1200.0,2.0,0.0,13000000.0,10833.333333,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
dff.shape

(8138, 247)

In [43]:
df.corr()

,bhk,total_area,bath,balcony,price,price_per_sqft
bhk,1.000000,0.634363,0.853028,0.192395,0.531016,0.236103
total_area,0.634363,1.000000,0.649762,0.144340,0.847678,0.277765
bath,0.853028,0.649762,1.000000,0.187310,0.580616,0.292715
balcony,0.192395,0.144340,0.187310,1.000000,0.117207,0.033033
price,0.531016,0.847678,0.580616,0.117207,1.000000,0.680268
price_per_sqft,0.236103,0.277765,0.292715,0.033033,0.680268,1.000000


### MOdel training

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

model = LinearRegression()

In [45]:
%%time
x= dff.drop(columns=['price','price_per_sqft'])
y = dff1['price']

CPU times: user 3.77 ms, sys: 4.21 ms, total: 7.98 ms
Wall time: 7.08 ms


In [46]:
y.shape

(8138,)

In [47]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.35,random_state=10)

In [48]:
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.8733565105010153

In [57]:
print('Model Acccuracy:',model.score(x_test,y_test)*100,'%')

Model Acccuracy: 87.33565105010153 %


### Predictions

### As we used hot encoding for th elocation so we need a script to take input as location and get desired predictions

Input Parameters: Bhk;total_sqft;bath;location

In [49]:
x.columns

Index(['bhk', 'total_area', 'bath', 'balcony', '1st Block Jayanagar',
       '1st Phase JP Nagar', '2nd Phase Judicial Layout',
       '2nd Stage Nagarbhavi', '5th Block Hbr Layout', '5th Phase JP Nagar',
       ...
       'Vijayanagar', 'Vishveshwarya Layout', 'Vishwapriya Layout',
       'Whitefield', 'Yelachenahalli', 'Yelahanka', 'Yelahanka New Town',
       'Yelenahalli', 'Yeshwanthpur', 'other'],
      dtype='object', length=245)

In [50]:
# Script for prediction
def prediction(location,sqft,bhk,bath):
    location_index = np.where(x.columns == location)[0][0]

    X = np.zeros(len(x.columns))
    X[0]=bhk
    X[1]=sqft
    X[2]=bath  
    #X[3]=price_per_sqft   
 # X[3]= pricepersqft'( if we want to add price_p_sqft as a feature)       
    if location_index>=0:
        X[location_index]=1
    return model.predict([X])[0]


In [51]:
#Give input in order of location,sqft,bhk,bath

Price_prediction = prediction('1st Phase JP Nagar',1400,2,2)

print("The predicted price is",Price_prediction ,"lakh")

The predicted price is 11864502.724425813 lakh


### Now i am Going to check other Regression models and find out which one is the best

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [53]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
#import xgboost as xgb 
from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor

def find_best_model_using_gridsearchcv(x , y):
    algos = {
        'LinearRegression' : {
            'model' : LinearRegression(),
            'params' : {
                'normalize' : [True , False],
                'fit_intercept': [True , False],
                 'copy_X' : [True , False]
            }
        },
        'lasso' : {
            'model' : Lasso(),
            'params' : {
                'alpha' : [1, 10, 50, 200, 500],
                'selection' : ['random' , 'cyclic']
            } 
        },
         'Ridge' : {
            'model' : Ridge(), 
            'params' : {
                'alpha' : [1, 10, 50, 200, 500],
                'fit_intercept' : [True , False],
                'normalize' : [True , False],
            }
        },
        'descision_tree' : {
            'model' : DecisionTreeRegressor(),
            'params' :{
                'criterion' : ['mse' , 'friedman_mse'],
                'splitter' : ['best' , 'random']
            }
        }
    }
    
    scores = []
    cv = ShuffleSplit(n_splits = 5 , test_size = 0.2 , random_state = 0)
    for algo_name , config in algos.items():
        gs = GridSearchCV(config['model'] , config['params'] , cv = cv , return_train_score = False)
        gs.fit(x,y)
        scores.append({
            'model' : algo_name , 
            'best_score' : gs.best_score_,
            'best_params' : gs.best_params_
        })
 
    return pd.DataFrame(scores , columns = ['model' , 'best_score' , 'best_params'])  

In [54]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

find_best_model_using_gridsearchcv(x,y) 

### Thats why we used Liner Reg model

{Project by : Kumar Shivam}